# Obtención de los Hyperparametros Optimizados para modelo de detección de Detectron2

**Importamos las librerias y modulos necesarios:**

In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_test_loader
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.engine import DefaultPredictor
from detectron2.structures import Boxes

from matplotlib import pyplot as plt
from PIL import Image
import optuna
import locale
import os
import json
import pandas as pd
import numpy as np
import yaml
import torch
import cv2
import random
from pathlib import Path
import time
import shutil
from matplotlib import pyplot as plt
import re
import time 

ModuleNotFoundError: No module named 'detectron2'

Asegurar que hay una gpu disponible

In [ ]:
print("CUDA disponible:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("Número de dispositivos CUDA disponibles:", torch.cuda.device_count())
    print("Nombre del dispositivo CUDA actual:", torch.cuda.get_device_name())

**Rutas del directorio donde se han guarrdad los modelos**

In [ ]:
ruta_resultados = "models/Final/Detectron2_s_20_all/"

**Juntamos todas las metricas en un solo archivo**

In [ ]:
# Contar el número de carpetas que siguen el patrón "trial_"
trial_folders = [folder for folder in os.listdir(ruta_resultados) if folder.startswith("trial_") and folder.split("_")[1].isdigit()]
n_trials = len(trial_folders)
print(n_trials)

# Crear una lista para almacenar todas las métricas
all_metrics = []

# Recorrer todas las carpetas de trials y leer los archivos JSON de métricas
for trial_num in range(n_trials):
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial_num}")
#     result_path = os.path.join(trial_dir, f"trial_{trial_num}_result.json")
    result_path = os.path.join(trial_dir, f"metrics.json")
    if os.path.exists(result_path):
        with open(result_path, 'r') as f:
            trial_metrics = json.load(f)
            trial_metrics["trial_number"] = trial_num  # Añadir el número del trial a las métricas
            all_metrics.append(trial_metrics)

# Guardar todas las métricas en un archivo JSON
all_metrics_path = os.path.join(ruta_resultados, "all_metrics.json")
with open(all_metrics_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"Métricas de todos los trials guardadas en {all_metrics_path}")


**Se verifica con que dataset fue entrenado el modelo**

# Leer el archivo all_metrics.json
with open(all_metrics_path, 'r') as f:
    all_metrics = json.load(f)

# Extraer el valor del campo "Dataset"
if all_metrics:
    data_yaml_path = all_metrics[1]["Dataset"]
else:
    data_yaml_path = None

# Extraer el valor del campo "Dataset"
if all_metrics:
    pretrained_weights = all_metrics[1]["Pre-Trained_Weights"]
else:
    pretrained_weights = None

print(f"El archivo YAML utilizado es: {data_yaml_path}")
print(f"Los pesos preentrenados usados son: {pretrained_weights}")


In [ ]:
def crear_directorio(directorio):
    """Crea un directorio si no existe."""
    print(directorio)
    if not os.path.exists(directorio):
        os.makedirs(directorio)
        print(f"Directorio '{directorio}' creado.")
    else:
        print(f"El directorio '{directorio}' ya existe.")

In [ ]:
def procesar_trial(trial_path, data_yaml_path):
    print("Iniciando procesar_trial")
    # Definir la ruta para la carpeta del trial y el archivo all_metrics.json
    model_path = os.path.join(trial_path, "model_final.pth")  # Archivo .pth para Detectron2
    print("\n\nCargado modelo: ", model_path)
    
    # Crear carpeta para coordenadas
    output_folder = os.path.join(trial_path, "coordenadas_test")
    os.makedirs(output_folder, exist_ok=True)
    print(f"Carpeta de salida creada: {output_folder}")
    
    # Modificar la ruta, subir dos directorios y añadir /test/annotations.json
    data_test_yaml_path = os.path.join(Path(data_yaml_path).parents[1], 'test', 'annotations.json')
    print(f"Ruta de anotaciones de prueba: {data_test_yaml_path}")
    # Obtener el directorio de test
    test_folder = Path(data_test_yaml_path).parent
    
    image_test_folder = os.path.join(test_folder, 'images')
    print(f"Carpeta de imágenes de prueba: {image_test_folder}")
    
    print("Configurando el modelo Detectron2")
    # Configuración del modelo Detectron2
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file(pretrained_weights))
    cfg.MODEL.WEIGHTS = model_path
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Umbral de confianza
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
    print(f"Dispositivo utilizado: {cfg.MODEL.DEVICE}")
    predictor = DefaultPredictor(cfg)
    
    # Lista para almacenar todas las predicciones
    all_predictions = []
    
    print("Iniciando procesamiento de imágenes")
    # Procesar imágenes
    for img_path in Path(image_test_folder).glob('*'):
        print(f"Procesando imagen: {img_path}")
        # Leer la imagen y hacer predicciones
        img = cv2.imread(str(img_path))
        if img is None:
            print(f"Error: No se pudo leer la imagen {img_path}")
            continue
        
        img_height, img_width = img.shape[:2]
        print(f"Tamaño de la imagen: {img_width}x{img_height}")
        
        print("Realizando predicción")
        outputs = predictor(img)
        
        # Extraer las predicciones: cajas, clases, puntuaciones y máscaras
        instances = outputs["instances"].to("cpu")
        pred_boxes = instances.pred_boxes.tensor.numpy() if instances.has("pred_boxes") else None
        pred_classes = instances.pred_classes.numpy() if instances.has("pred_classes") else None
        scores = instances.scores.numpy() if instances.has("scores") else None
        pred_masks = instances.pred_masks.numpy() if instances.has("pred_masks") else None
        
        print(f"Número de predicciones: {len(pred_boxes) if pred_boxes is not None else 0}")
        
        # Crear predicciones en formato Detectron2
        for i in range(len(pred_boxes)):
            prediction = {
                "image_id": img_path.stem,
                "category_id": int(pred_classes[i]),
                "bbox": pred_boxes[i].tolist(),
                "score": float(scores[i])
            }
            if pred_masks is not None:
                # Convertir la máscara a coordenadas
                mask_coords = mask_to_coordinates(pred_masks[i], img_width, img_height)
                print(f"Formato de coordenadas de la máscara {i} para {img_path.stem}:")
                print(mask_coords)
                
                # Guardar las coordenadas en el JSON
                prediction["segmentation"] = mask_coords
            
            all_predictions.append(prediction)
        
        print(f"Predicciones procesadas para: {img_path}")
        print("-" * 40)
    
    print(f"Total de predicciones: {len(all_predictions)}")
    
    # Guardar todas las predicciones en un único archivo JSON
    output_file = os.path.join(output_folder, "predictions.json")
    with open(output_file, 'w') as f:
        json.dump(all_predictions, f, indent=2)
    
    print(f"Todas las predicciones guardadas en: {output_file}")

def mask_to_coordinates(mask, img_width, img_height):
    """
    Convert a binary mask to a list of (coor_X, coor_Y) coordinates.
    """
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    coordinates = []
    for contour in contours:
        for point in contour:
            x, y = point[0]
            # Asegurarse de que las coordenadas estén dentro del rango de la imagen
            x = min(max(x, 0), img_width - 1)
            y = min(max(y, 0), img_height - 1)
            coordinates.extend([float(x), float(y)])
    return coordinates

In [ ]:
def borrar_coordenadas_test(trial_path):
    """Eliminar la carpeta coordenadas_test si existe dentro de trial_path."""
    coordenadas_test_path = os.path.join(trial_path, "coordenadas_test")
    if os.path.exists(coordenadas_test_path) and os.path.isdir(coordenadas_test_path):
        shutil.rmtree(coordenadas_test_path)
        print(f"Carpeta 'coordenadas_test' eliminada en: {trial_path}")
    else:
        print(f"No se encontró la carpeta 'coordenadas_test' en: {trial_path}")
        
# Iterar sobre cada carpeta trial y borrar coordenadas_test
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        borrar_coordenadas_test(trial_path)   

In [ ]:
# Iterar sobre cada carpeta trial y procesar
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        procesar_trial(trial_path,data_yaml_path)

In [ ]:
def aplanar_segmentacion(segmentacion):
    """
    Aplana la lista de segmentación de coordenadas, eliminando corchetes anidados.
    """
    return [coordenadas for grupo in segmentacion for coordenadas in grupo]

def evaluar_trial(trial_path, data_yaml_path):
    # Construir las rutas de los archivos JSON
    ground_truths_path = os.path.join(Path(data_yaml_path).parents[1], 'test', 'annotations.json')
    predictions_path = os.path.join(trial_path, "coordenadas_test", "predictions.json")
    
    print(f"Ruta de ground_truths: {ground_truths_path}")
    print(f"Ruta de predictions: {predictions_path}")
    
    # Cargar los archivos JSON
    try:
        with open(ground_truths_path) as f:
            ground_truths = json.load(f)
        print("Cargado ground_truths.json exitosamente.")
    except Exception as e:
        print(f"Error al cargar ground_truths.json: {e}")
        return

    try:
        with open(predictions_path) as f:
            predictions = json.load(f)
        print("Cargado predictions.json exitosamente.")
    except Exception as e:
        print(f"Error al cargar predictions.json: {e}")
        return

    # Crear un diccionario para mapear file_name (sin extensión) a image_id en ground_truths
    file_name_to_image_id = {os.path.splitext(img['file_name'])[0]: img['id'] for img in ground_truths.get('images', [])}
    print(f"Diccionario file_name_to_image_id: {file_name_to_image_id}")

    # Crear un diccionario para mapear image_id a sus anotaciones en ground_truths
    annotations_by_image = {}
    image_info = {}  # Nuevo diccionario para guardar información de altura y anchura

    for img in ground_truths.get('images', []):
        image_id = img['id']
        image_info[image_id] = {'file_name': img['file_name'], 'height': img['height'], 'width': img['width']}
    
    for ann in ground_truths.get('annotations', []):
        image_id = ann['image_id']
        if image_id not in annotations_by_image:
            annotations_by_image[image_id] = {'segmentation_GT': []}
        annotations_by_image[image_id]['segmentation_GT'].append(ann['segmentation'])
    
#     print(f"Diccionario annotations_by_image: {annotations_by_image}")
#     print(f"Diccionario image_info: {image_info}")

    # Crear un diccionario para mapear image_id (file_name en predictions) a sus predicciones en predictions
    predictions_by_image = {}
    for pred in predictions:
        file_name = pred['image_id']
        if file_name in file_name_to_image_id:
            image_id = file_name_to_image_id[file_name]
            if image_id not in predictions_by_image:
                predictions_by_image[image_id] = {'category_id': pred['category_id'], 'segmentation_Pred': []}
            predictions_by_image[image_id]['segmentation_Pred'].append(pred['segmentation'])
        else:
            print(f"Advertencia: El file_name '{file_name}' de predictions no se encuentra en ground_truths.")

#     print(f"Diccionario predictions_by_image: {predictions_by_image}")

    # Emparejar las coordenadas de segmentación
    results = []
    for image_id, anns in annotations_by_image.items():
        if image_id in predictions_by_image:
            file_name = image_info[image_id]['file_name']
            result = {
                'file_name': file_name,
                'category_id': predictions_by_image[image_id]['category_id'],
                'height': image_info[image_id]['height'],
                'width': image_info[image_id]['width'],
                'segmentation_GT': aplanar_segmentacion(anns['segmentation_GT']),
                'segmentation_Pred': aplanar_segmentacion(predictions_by_image[image_id]['segmentation_Pred'])
                
            }
            results.append(result)
    
    print(f"Resultados emparejados: {results}")

    # Guardar el resultado en un nuevo archivo JSON
    output_path = os.path.join(trial_path, "coordenadas_test/matched_segments.json")
    try:
        with open(output_path, 'w') as f:
            json.dump(results, f, indent=2)
        print(f"El emparejamiento de coordenadas de segmentación se ha guardado en '{output_path}'.")
    except Exception as e:
        print(f"Error al guardar matched_segments.json: {e}")

# Iterar sobre cada carpeta trial y procesar
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        evaluar_trial(trial_path, data_yaml_path)


In [ ]:
# Función para dibujar las máscaras en una imagen de fondo negro
def draw_masks_on_black_background(gt_mask, pred_mask, img_width, img_height):
    # Crear una imagen negra
    black_background = np.zeros((img_height, img_width, 3), dtype=np.uint8)
    
    # Crear máscaras de colores
    gt_colored = np.stack([gt_mask * 255, np.zeros_like(gt_mask), np.zeros_like(gt_mask)], axis=-1)  # Azul
    pred_colored = np.stack([np.zeros_like(pred_mask), np.zeros_like(pred_mask), pred_mask * 255], axis=-1)  # Rojo
    
    # Superponer las máscaras en la imagen negra con transparencia
    combined_image = cv2.addWeighted(black_background, 1, gt_colored, 0.5, 0)
    combined_image = cv2.addWeighted(combined_image, 1, pred_colored, 0.5, 0)
    
    return combined_image

# Función para dibujar contornos de las máscaras en la imagen original
def draw_contours_on_image(image, gt_mask, pred_mask):
    contours_gt, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_pred, _ = cv2.findContours(pred_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_contours = image.copy()
    cv2.drawContours(image_contours, contours_gt, -1, (255, 0, 0), 2)  # Azul para GT
    cv2.drawContours(image_contours, contours_pred, -1, (0, 0, 255), 2)  # Rojo para Pred
    
    return image_contours

# Función para crear máscara binaria a partir de las coordenadas
def create_binary_mask(coords_list, img_width, img_height):
    mask = np.zeros((img_height, img_width), dtype=np.uint8)
    if isinstance(coords_list[0], (int, float)):
        coords_list = [coords_list]  # Envolver en una lista para uniformidad
    
    for coords in coords_list:
        coords = np.array(coords)
        if coords.ndim == 1:
            coords = coords.reshape(-1, 2)
        coords = coords.astype(np.int32)
        
        if coords.shape[0] > 0:
            cv2.fillPoly(mask, [coords], 1)
    
    return mask

# Modificar la función evaluar_trial para usar la nueva función
def evaluar_trial(trial_path, dataset_path):
    # Extraer número del trial
    trial_number = extract_trial_number(trial_path)
    if trial_number is None:
        raise ValueError(f"No se pudo extraer el número del trial de la ruta: {trial_path}")

    # Definir la ruta del archivo matched_segments.json
    json_file = os.path.join(trial_path, "coordenadas_test", "matched_segments.json")
    
    # Cargar el archivo JSON
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Seleccionar 3 archivos aleatorios
    random_files = random.sample(data, min(3, len(data)))

    results = []
    
    # Procesar cada imagen en el archivo JSON
    for item in data:
        file_name = item["file_name"]
        if not file_name.endswith(".jpg"):
            file_name += ".jpg"  # Añadir la extensión si no está presente

        img_width, img_height = item["width"], item["height"]
        segmentation_gt = item.get('segmentation_GT', [])
        segmentation_pred = item.get('segmentation_Pred', [])

        # Cargar la imagen original
        image_path = os.path.join(dataset_path, "test/images", file_name)
        image = cv2.imread(image_path)
        if image is None:
            print(f"Error: No se pudo cargar la imagen {image_path}. Verifica la ruta y el archivo.")
            continue  # Saltar esta imagen si no se carga correctamente

        # Crear máscaras binarias a partir de las coordenadas
        gt_mask = create_binary_mask(segmentation_gt, img_width, img_height)
        pred_mask = create_binary_mask(segmentation_pred, img_width, img_height)
        
        # Binarizar las máscaras
        binary_true_mask = (gt_mask > 0).astype(int)
        binary_predicted_mask = (pred_mask > 0).astype(int)
        
        # Calcular TP, TN, FP, FN
        TP = np.sum((binary_predicted_mask == 1) & (binary_true_mask == 1))
        FP = np.sum((binary_predicted_mask == 1) & (binary_true_mask == 0))
        TN = np.sum((binary_predicted_mask == 0) & (binary_true_mask == 0))
        FN = np.sum((binary_predicted_mask == 0) & (binary_true_mask == 1))
        
        results.append([file_name, TP, TN, FP, FN])

        # Generar figuras para las 3 imágenes seleccionadas
        if item in random_files:
            # Dibujar máscaras en fondo negro
            combined_image = draw_masks_on_black_background(gt_mask, pred_mask, img_width, img_height)
            
            # Guardar la imagen combinada
            figuras_folder = os.path.join(trial_path, 'Figuras')
            if not os.path.exists(figuras_folder):
                os.makedirs(figuras_folder)

            combined_image_file = os.path.join(figuras_folder, f"{file_name}_mask_eval.png")
            cv2.imwrite(combined_image_file, combined_image)

            # Dibujar contornos de las máscaras en la imagen original
            contour_image = draw_contours_on_image(image, gt_mask, pred_mask)
            visual_image_file = os.path.join(figuras_folder, f"{file_name}_mask_visual.png")
            cv2.imwrite(visual_image_file, contour_image)

    # Guardar resultados
    results_df = pd.DataFrame(results, columns=['Filename', 'TP', 'TN', 'FP', 'FN'])
    mean_TP = results_df['TP'].mean()
    mean_TN = results_df['TN'].mean()
    mean_FP = results_df['FP'].mean()
    mean_FN = results_df['FN'].mean()
    
    precision_avg = mean_TP / (mean_TP + mean_FP) if (mean_TP + mean_FP) > 0 else 0
    recall_avg = mean_TP / (mean_TP + mean_FN) if (mean_TP + mean_FN) > 0 else 0
    f1_score_avg = (2 * precision_avg * recall_avg) / (precision_avg + recall_avg) if (precision_avg + recall_avg) > 0 else 0
    iou_avg = mean_TP / (mean_TP + mean_FP + mean_FN) if (mean_TP + mean_FP + mean_FN) > 0 else 0
    
    metrics = {
        'mean_TP': mean_TP,
        'mean_TN': mean_TN,
        'mean_FP': mean_FP,
        'mean_FN': mean_FN,
        'precision_avg': precision_avg,
        'recall_avg': recall_avg,
        'f1_score_avg': f1_score_avg,
        'iou_avg': iou_avg
    }
    
    metrics_file = os.path.join(trial_path, f"trial_{trial_number}_metricas.json")
    with open(metrics_file, 'w') as f:
        json.dump(metrics, f, indent=4)

    print(f"Resultados guardados en {metrics_file}")

# Función para extraer el número de trial
def extract_trial_number(trial_path):
    import re
    match = re.search(r"trial_(\d+)", trial_path)
    if match:
        return int(match.group(1))  # Devuelve el número como entero
    return None


In [ ]:
def borrar_Figuras(trial_path):
    """Eliminar la carpeta Figuras si existe dentro de trial_path."""
    Figuras_path = os.path.join(trial_path, "Figuras")
    if os.path.exists(Figuras_path) and os.path.isdir(Figuras_path):
        shutil.rmtree(Figuras_path)
        print(f"Carpeta 'Figuras' eliminada en: {trial_path}")
    else:
        print(f"No se encontró la carpeta 'Figuras' en: {trial_path}")
        
# Iterar sobre cada carpeta trial y borrar Figuras
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        borrar_Figuras(trial_path)

In [ ]:
dataset_path = Path(Path(data_yaml_path).parent).parent

# Iterar sobre cada carpeta trial y procesar
for folder in os.listdir(ruta_resultados):
    trial_path = os.path.join(ruta_resultados, folder)
    if os.path.isdir(trial_path) and folder.startswith("trial_") and folder.split("_")[1].isdigit():
        evaluar_trial(trial_path,dataset_path)

**Juntamos todos los json de las metricas**

In [ ]:
# Contar el número de carpetas que siguen el patrón "trial_"
trial_folders = [folder for folder in os.listdir(ruta_resultados) if folder.startswith("trial_") and folder.split("_")[1].isdigit()]
n_trials = len(trial_folders)

# Crear una lista para almacenar todas las métricas
all_metrics = []

# Recorrer todas las carpetas de trials y leer los archivos JSON de métricas
for trial_num in range(n_trials):
    trial_dir = os.path.join(ruta_resultados, f"trial_{trial_num}")
    metrics_path = os.path.join(trial_dir, f"trial_{trial_num}_metricas.json")
    if os.path.exists(metrics_path):
        with open(metrics_path, 'r') as f:
            trial_metrics = json.load(f)
            trial_metrics["trial_number"] = trial_num  # Añadir el número del trial a las métricas
            all_metrics.append(trial_metrics)

# Guardar todas las métricas en un archivo JSON
evaluacion_path = os.path.join(ruta_resultados, "evaluacion.json")
with open(evaluacion_path, 'w') as f:
    json.dump(all_metrics, f, indent=2)

print(f"Métricas de todos los trials guardadas en {evaluacion_path}")

**Creamos un criterio por el que obtenemos el mejor modelo de estos trials, segun la media de sus métricas**

In [ ]:
# Leer el archivo JSON
with open(evaluacion_path, 'r') as f:
    metrics_data = json.load(f)

# Definir pesos para cada métrica (puedes ajustar los pesos según tu criterio)
weights = {
    "precision_avg": 0.25,
    "recall_avg": 0.25,
    "f1_score_avg": 0.25,
    "iou_avg": 0.25
}

# Crear una lista con los trials y sus medias ponderadas
trials_with_scores = []

for trial in metrics_data:
    precision = trial.get("precision_avg", 0)
    recall = trial.get("recall_avg", 0)
    f1_score = trial.get("f1_score_avg", 0)
    iou = trial.get("iou_avg", 0)
    
    # Calcular la media ponderada
    weighted_score = (
        weights["precision_avg"] * precision +
        weights["recall_avg"] * recall +
        weights["f1_score_avg"] * f1_score +
        weights["iou_avg"] * iou
    )
    
    # Añadir a la lista
    trials_with_scores.append({
        "trial_number": trial["trial_number"],
        "weighted_score": weighted_score
    })

# Ordenar los trials por la media ponderada de mayor a menor
sorted_trials = sorted(trials_with_scores, key=lambda x: x["weighted_score"], reverse=True)

# Mostrar la lista ordenada
for trial in sorted_trials:
    print(f"Trial {trial['trial_number']}: Weighted Score = {trial['weighted_score']}")

# Si quieres guardar el resultado en un archivo JSON
sorted_trials_path = os.path.join(os.path.dirname(evaluacion_path), "sorted_trials.json")
with open(sorted_trials_path, 'w') as f:
    json.dump(sorted_trials, f, indent=2)

print(f"Lista de trials ordenados guardada en {sorted_trials_path}")


**Creamos un criterio por el que obtenemos el mejor modelo de estos trials, segun el teimpo de inferencia**

In [ ]:
    # Inicializar variables
    resultados_inferencia = []

    # Obtener la ruta de las imágenes originales
    dataset_path = os.path.dirname(data_yaml_path)
    dataset_test_labels = os.path.join(dataset_path, 'test/labels')
    ground_truths = dataset_test_labels
    dataset_test_path = os.path.dirname(ground_truths)
    ruta_imagenes_test = os.path.join(dataset_test_path, 'images')

    # Obtener todas las carpetas de trials
    for trial_folder in os.listdir(ruta_resultados):
        trial_path = os.path.join(ruta_resultados, trial_folder)

        # Verificar si la carpeta es un trial
        if os.path.isdir(trial_path) and trial_folder.startswith("trial_"):
            trial_number = int(trial_folder.split("_")[1])

            # Cargar el modelo Detectron2 del trial
            config_file = os.path.join(trial_path, "config.yaml")  # Ruta al archivo de configuración
            model_path = os.path.join(trial_path, "model_final.pth")  # Ruta al archivo de pesos

            # Configuración del modelo Detectron2
            cfg = get_cfg()
            cfg.merge_from_file(model_zoo.get_config_file(pretrained_weights))
            cfg.MODEL.WEIGHTS = model_path
            cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Umbral de confianza
            cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
            cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
            print(f"Dispositivo utilizado: {cfg.MODEL.DEVICE}")
            predictor = DefaultPredictor(cfg)

            predictor = DefaultPredictor(cfg)
            print(f"\nModelo cargado para el trial {trial_number}")

            # Obtener las imágenes de test
            test_folder = os.path.join(Path(data_yaml_path).parents[1], 'test', 'images')
            print(f"test_folder: ", test_folder)
            image_paths = list(Path(test_folder).glob('*'))

            # Inicializar el contador de tiempo
            total_inference_time = 0
            num_images = len(image_paths)

            # Realizar la inferencia sobre las imágenes y medir el tiempo
            for img_path in image_paths:
                print(f"Procesando imagen: {img_path}")
                start_time = time.time()

                # Leer la imagen
                image = cv2.imread(str(img_path))
                if image is None:
                    print(f"Error: No se pudo cargar la imagen {img_path}.")
                    continue

                # Ejecutar inferencia
                outputs = predictor(image)

                # Calcular tiempo de inferencia
                inference_time = time.time() - start_time
                total_inference_time += inference_time

                # Mostrar el tiempo de inferencia para cada imagen
                print(f"Tiempo de inferencia para {img_path}: {inference_time:.4f} segundos")

            # Calcular el tiempo de inferencia promedio
            avg_inference_time = total_inference_time / num_images if num_images > 0 else 0

            # Guardar el resultado
            resultados_inferencia.append({
                "trial_number": trial_number,
                "inference_time": avg_inference_time
            })

    # Ordenar los resultados por tiempo de inferencia (menor a mayor)
    resultados_inferencia.sort(key=lambda x: x["inference_time"])

    # Guardar los resultados en un archivo JSON
    inference_times_path = os.path.join(ruta_resultados, "inference_times.json")
    with open(inference_times_path, 'w') as f:
        json.dump(resultados_inferencia, f, indent=4)

    print(f"Resultados guardados en {inference_times_path}")


**Creamos un .json con todas las métricas: Resultados.json**

In [ ]:
# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

with open(inference_times_path, 'r') as f:
    inference_times_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Crear un diccionario para acceder rápidamente a la información de inference_times
inference_times_dict = {trial['trial_number']: trial['inference_time'] for trial in inference_times_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados = []

# Ordenar los trial_numbers basados en sorted_trials_dict
sorted_trial_numbers = sorted(sorted_trials_dict.keys())

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    inference_time = inference_times_dict.get(trial_number, 0)  # Obtener el tiempo de inferencia

    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number],
        "inference_time": inference_time  # Añadir el tiempo de inferencia
    }

    # Añadir la información común y la información del trial a la lista de resultados
    resultados.append({**common_info, **trial_info})

# Guardar el resultado en el archivo Resultados.json
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados, f, indent=2)

print(f"Archivo Resultados.json actualizado con tiempos de inferencia guardado en {resultados_path}")


**Resultados ordenados por las mejores metricas**

In [ ]:
# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

with open(inference_times_path, 'r') as f:
    inference_times_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Crear un diccionario para acceder rápidamente a la información de inference_times
inference_times_dict = {trial['trial_number']: trial['inference_time'] for trial in inference_times_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados_ordenados = []

# Extraer los trial_numbers en el orden exacto en el que aparecen en sorted_trials_data
sorted_trial_numbers = [trial['trial_number'] for trial in sorted_trials_data]

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    inference_time = inference_times_dict.get(trial_number, 0)  # Obtener el tiempo de inferencia

    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number],
        "inference_time": inference_time  # Añadir el tiempo de inferencia
    }

    # Añadir la información común y la información del trial a la lista de resultados
    resultados_ordenados.append({**common_info, **trial_info})

# Guardar el resultado en el archivo con un nombre dinámico
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados_ordenados_metricas.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados_ordenados, f, indent=2)

print(f"Archivo Resultados_ordenados_metricas.json guardado en {resultados_path}")


**Resultados ordenados por los mejores tiempos de inferencia**

In [ ]:
# Leer los archivos JSON
with open(evaluacion_path, 'r') as f:
    evaluacion_data = json.load(f)

with open(sorted_trials_path, 'r') as f:
    sorted_trials_data = json.load(f)

with open(all_metrics_path, 'r') as f:
    all_metrics_data = json.load(f)

with open(inference_times_path, 'r') as f:
    inference_times_data = json.load(f)

# Crear un diccionario para acceder rápidamente a la información de sorted_trials
sorted_trials_dict = {trial['trial_number']: trial['weighted_score'] for trial in sorted_trials_data}

# Crear un diccionario para acceder rápidamente a la información de all_metrics
all_metrics_dict = {trial['Trial']: trial for trial in all_metrics_data}

# Crear un diccionario para acceder rápidamente a la información de inference_times
inference_times_dict = {trial['trial_number']: trial['inference_time'] for trial in inference_times_data}

# Obtener la información común del primer elemento de all_metrics_data
if all_metrics_data:
    common_info = {
        "Dataset": all_metrics_data[0].get("Dataset", ""),
        "Feedback": all_metrics_data[0].get("Feedback", ""),
        "Pre-Trained_Weights": all_metrics_data[0].get("Pre-Trained_Weights", "")
    }
else:
    common_info = {
        "Dataset": "",
        "Feedback": "",
        "Pre-Trained_Weights": ""
    }

# Crear el archivo de salida
resultados_ordenados = []

# Extraer los trial_numbers en el orden exacto en el que aparecen en sorted_trials_data
sorted_trial_numbers = [trial['trial_number'] for trial in sorted_trials_data]

# Combinar la información
for trial_number in sorted_trial_numbers:
    # Buscar la información de evaluación y métricas
    evaluation_info = next((item for item in evaluacion_data if item["trial_number"] == trial_number), {})
    metrics_info = all_metrics_dict.get(trial_number, {})
    inference_time = inference_times_dict.get(trial_number, 0)  # Obtener el tiempo de inferencia

    # Crear el objeto para el trial actual
    trial_info = {
        "trial_number": trial_number,
        "hyperparameters": metrics_info.get("hyperparameters", {}),
        "metrics": metrics_info.get("metrics", {}),
        "evaluation": evaluation_info,
        "weighted_score": sorted_trials_dict[trial_number],
        "inference_time": inference_time  # Añadir el tiempo de inferencia
    }

    # Añadir la información común y la información del trial a la lista de resultados
    resultados_ordenados.append({**common_info, **trial_info})

# Ordenar los resultados por tiempo de inferencia (menor a mayor)
resultados_ordenados.sort(key=lambda x: x["inference_time"])

# Guardar el resultado en el archivo con un nombre dinámico
resultados_path = os.path.join(os.path.dirname(trial_dir), "Resultados_ordenados_time.json")
with open(resultados_path, 'w') as f:
    json.dump(resultados_ordenados, f, indent=2)

print(f"Archivo Resultados_ordenados_metricas.json guardado en {resultados_path}")
